In [61]:
# Standard Imports
from __future__ import print_function
import numpy as np
import pandas as pd
import os
import sys
from collections import defaultdict
from importlib import reload
from bs4 import BeautifulSoup
import requests
import scipy.stats as scs
import time
import seaborn as sns

import matplotlib.pyplot as plt

# Modeling
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import datasets, layers, models
# import keras.backend as K
# from tensorflow.keras.constraints import min_max_norm, non_neg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

# Load MongoDB
from pymongo import MongoClient
client = MongoClient()
# Access/Initiate Database
db = client['producer_db']
# Access/Initiate Table
mfcc_tab = db['mfcc']
tab = db['songs']
collection = db.tab
mfcc_collection = db.mfcc_tab

# Authorize Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Discogs token
discogs_token = os.environ['DISCOGS_TOKEN']

# Plotly
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import colorlover as cl
from IPython.display import HTML
plotly_username = os.environ['PLOTLY_USERNAME']
plotly_api_key = os.environ['PLOTLY_API_KEY']
plotly.tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key)
#set colormap
Set3_10 = cl.scales['10']['qual']['Set3']
cm_10 = list(zip(np.linspace(0,1,10),Set3_10)) 

# Custom Modules
import src.knn_modeling as km

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

# Audio Processing
import librosa
import librosa.display
from IPython.display import Audio
import IPython.display
from scipy.io import wavfile
from pydub import AudioSegment
from src.audio_processing import load_mp3_from_url

In [74]:
reload(km)
pv = km.ProductionValue(collection = mfcc_collection, sp = sp, discogs_token = discogs_token)
pv.fit_knn('mfcc_highpass')

# Production Value: Demo

The `demo` below takes values for a `track` (name of the song) and optional values for the `artist` and `album` for **any song** and will return likely producers and the most similar songs in the database.

In [76]:
_, songs, _ = pv.demo(track = 'sound and color', artist='alabama shakes')
songs

sound and color artist:alabama shakes
True Producer: Alabama Shakes

Producer Probabilities:
[['Pete Rock' 0.43333333333333335]
 ['RZA' 0.13333333333333333]
 ['Kanye West' 0.13333333333333333]
 ['George Martin' 0.1]
 ['Brian Eno' 0.1]
 ['Paul Epworth' 0.06666666666666667]
 ['Roy Thomas Baker' 0.03333333333333333]
 ['Stock Aitken Waterman' 0.0]
 ['Rick Rubin' 0.0]
 ['Dr. Dre' 0.0]]

Query Song: sound and color by alabama shakes



Most Similar Song: I've Got The World On A String by CL Smooth


,track,artist,album,producer,distance
625,I've Got The World On A String,CL Smooth,American Me,Pete Rock,28.069855
759,Lean on Me,Chris Brown,Before the Party,Kanye West,30.207559
627,My Foolish Heart,CL Smooth,American Me,Pete Rock,31.212607
841,Necklace,Wu-Tang Clan,A Better Tomorrow,RZA,34.328517
321,Paradise,Coldplay,Mylo Xyloto,Brian Eno,35.105801
